# Scraping posts from r/conspiracy

Extraction of all the posts resent in r/conspiracy, with title, content and url.
First of all we need to import praw and access reddit:

In [ ]:
import praw
from praw.models import MoreComments
import pandas as pd
reddit = praw.Reddit(client_id = "", client_secret= "", user_agent= "")

We can now access the subreddit, extract all the posts url in it and check which other subreddit reposted that posts and how much:

In [ ]:
import pandas as pd
post_list=list()
subreddit_list = list()
conspiracy_dict=dict()

for i in reddit.subreddit("conspiracy").top(limit=5000): #recupera i 5000 post top di conspiracy
    post_list.append((i.title, i.score, i.url)) #recupera titolo, numero di upvote + il permalink


for post in post_list:
    for repost in reddit.subreddit('all').search('url:'+post[2]): #trova quando il post è stato citato
        subreddit_url = str(repost.subreddit)
        subreddit_url = "https://www.reddit.com/r/" + subreddit_url
        if subreddit_url in conspiracy_dict.keys():
            conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
            conspiracy_dict[subreddit_url][1][0] +=1
        else:
            conspiracy_dict[subreddit_url]=[[],[1]]
            conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
            
                


df = pd.DataFrame(conspiracy_dict)
df.to_csv(r'results/conspiracy_data/conspiracy_top_url.csv',index=False)




# Creating the network
Now we need to create the network of subreddits involved.

In [ ]:
def get_all_in_dir(dir):
    for filename in os.listdir(dir):
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f[-4:] == ".csv":
            yield f

In [ ]:
import csv
import pandas as pd

datasets= [file for file in get_all_in_dir("results/1st_level")]
datasets.extend(file for file in get_all_in_dir("results/2nd_level"))

network_list = list()

for subr in datasets:
        #print(f'now opening {subr}')
        try:
                df1 = pd.read_csv(subr)
        except:
                pass
        try:
                df1 = pd.read_csv(subr, encoding='utf8')
        except:
                print(f'unable to open {subr}')
                continue
        sub_name=subr[subr.find("\\")+1:-4]
        for column in df1.columns:
                if column [:7] != "Unnamed":
                        col_name = column[column.find("/r/")+3:]
                        try:
                                network_list.append((sub_name, col_name, df1[column][1]))
                        except Exception as E:
                                print(E) 
                else: 
                        continue
print(network_list)

'''with open('Network.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(["Source", "Target", "Weight"])
    write.writerows(network_list)'''

# Looking for subnetworks in the original csvs
Now that we have divided the main network in subnetworks we can llok for them in the original files.

In [ ]:
def get_all_in_dir(dir):
    for filename in os.listdir(dir):
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f[-4:] == ".csv":
            yield f

In [ ]:
import csv
import pandas as pd

datasets= [file for file in get_all_in_dir("results/1st_level")]
datasets.extend(file for file in get_all_in_dir("results/2nd_level"))

networks=[file for file in get_all_in_dir("results/subnetworks/mixed")]

for net in networks:
        try:
                df_network = pd.read_csv(net)
        except:
                pass
        try:
                df_network = pd.read_csv(net, encoding='utf8')
        except:
                print(f'unable to open {subr}')
                continue
        network_name=net[net.find("mixed\\")+6:-4]
        network_name=network_name+"_done"
        network_list=list()
        for subr in datasets:
                #print(f'now opening {subr}')
                try:
                        df_dataset = pd.read_csv(subr)
                except:
                        pass
                try:
                        df_dataset = pd.read_csv(subr, encoding='utf8')
                except:
                        print(f'unable to open {subr}')
                        continue
                sub_name=subr[subr.find("\\")+1:-4]
                for column in df_dataset.columns:
                        if column [:7] != "Unnamed":
                                col_name = column[column.find("/r/")+3:]
                                for index, row in df_network.iterrows():
                                        if (col_name == row["Source"] and sub_name == row["Target"]) or (col_name == row["Target"] and sub_name == row["Source"]):
                                                try:
                                                        network_list.append((sub_name, col_name, df_dataset[column][1],df_dataset[column][0]))
                                                except Exception as E:
                                                        print(E) 
                                        else: 
                                                continue
        
        
        try:
                with open(network_name+'.csv', 'w', encoding='utf-8') as f:
        # using csv.writer method from CSV package
                        write = csv.writer(f)
                        write.writerow(["Source", "Target", "Weight","Comments"])
                        write.writerows(network_list)
        except Exception as E:
                print(E)

                
                        

# Topic modeling

# Cleaning the CSVs

In [24]:
to_clean= [file for file in get_all_in_dir("results/subnetworks/mixed")]
for file in to_clean:
    df=pd.read_csv(file, encoding="utf-8", dtype=str)
    df=df.drop(df.columns[4:], axis=1)
    df=df.dropna()
    df.to_csv(file)


# Retrieving comments

In [1]:
import praw
from praw.models import MoreComments
import pandas as pd
reddit = praw.Reddit(client_id = "", client_secret= "", user_agent= "")

In [2]:
def get_all_in_dir(dir):
    for filename in os.listdir(dir):
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f[-4:] == ".csv":
            yield f

In [4]:
import random
def iterSample(iterable, samplesize):
    results = []
    for i, v in enumerate(iterable):
        r = random.randint(0, i)
        if r < samplesize:
            if i < samplesize:
                results.insert(r, v) # add first samplesize items in random order
            else:
                results[r] = v # at a decreasing rate, replace random items


    return results

Here we import the dataset and the comments from the posts in common between subreddits.

In [5]:
import pandas as pd
import re
to_scan= [file for file in get_all_in_dir("results/subnetworks/mixed")]
for subreddit in to_scan:
        comment_dict = dict()
        print('Opening', subreddit)
        try: 

                df = pd.read_csv(subreddit, encoding='utf8', on_bad_lines='skip')
                
                if os.path.isfile('comments_'+subreddit.split('\\')[1]):
                        start = (pd.read_csv('comments_'+subreddit.split('\\')[1])['Source'].iloc[-1], pd.read_csv('comments_'+subreddit.split('\\')[1])['Target'].iloc[-1])
                        start= int(df.loc[(df['Source'] == start[0]) & (df['Target'] == start[1])].index[0])
                        if start >= len(df):
                                continue        
                        df = df[start+1:].dropna()
        except Exception as e:
               print('error',e,subreddit)
               continue
                
        for row in df.to_dict(orient='records'):
                if isinstance(row['Source'], float) or isinstance(row['Target'],float):
                        continue
                if 'https' in row['Source'] or 'https' in row['Target']:
                        continue
                try:
                        list_post= row['Posts'][1:-1].split(',')
                except Exception as e:
                        continue
                result = ''
                num = 0
                for post_url in list_post:
                        if post_url == 0:
                                continue
                        post_url = post_url.replace(' ','')
                        #tmp =  re.sub("\[\]\'", "", post_url) NON FUNZIONA E NON CAPISCO PERCHé
                        if post_url[1:-1] in comment_dict.keys():
                                result += comment_dict[post_url[1:-1]][0]
                                num += comment_dict[post_url[1:-1]][1]
                                continue
                        try:
                                post = reddit.submission(url=post_url[1:-1])
                                try:
                                        #This allows up to 1 reply to each post.
                                        post.comments.replace_more(limit=1)
                                except Exception as e:
                                        print(e)
                                        print(f'{post}s comments was not accessible')
                                        continue
                                num += len(post.comments)
                                this_comment = ''
                                if num > 500:
                                        for comment in iterSample(post.comments, 500):
                                                try:
                                                        this_comment += comment.body
                                                        for reply in comment.replies:
                                                                this_comment+= reply.body
                                                        this_comment=this_comment.replace(',','')
                                                        this_comment=this_comment.replace('\n','\s')
                                                except Exception as e:
                                                        print(e)
                                                        continue
                                else:
                                        for comment in post.comments:
                                                try:
                                                        this_comment += comment.body
                                                        for reply in comment.replies:
                                                                this_comment+= reply.body
                                                        this_comment=this_comment.replace(',','')
                                                        this_comment=this_comment.replace('\n','\s')

                                                except Exception as e:
                                                        print(e)
                                                        continue
                                comment_dict[post_url[1:-1]] = (this_comment,num)
                                result += this_comment
                        except Exception as e:
                                print(e)
                                print(post.permalink + ' is not accessible')
                                continue                                         
                res=pd.DataFrame.from_dict([{'Source':row['Source'], 'Target':row['Target'],'NumberComments': num,'Comments':result}])
                if os.path.isfile('comments_'+subreddit.split('\\')[1]):
                        res.to_csv('comments_'+subreddit.split('\\')[1],  encoding='utf8', mode='a', header=False)
                else:
                        res.to_csv('comments_'+subreddit.split('\\')[1],  encoding='utf8')
                        
        print(subreddit, ' is finished')
                        
                        

Opening results/subnetworks/mixed\bitcoins_metareddit_news.csv
error index 0 is out of bounds for axis 0 with size 0 results/subnetworks/mixed\bitcoins_metareddit_news.csv
Opening results/subnetworks/mixed\left_right_bernie.csv
results/subnetworks/mixed\left_right_bernie.csv  is finished
Opening results/subnetworks/mixed\sciencePseudoscience_conspiracies.csv
results/subnetworks/mixed\sciencePseudoscience_conspiracies.csv  is finished
